In [1]:
import torch
import torch.nn.init as init
import torch.nn as nn

In [2]:
## バージョンの確認
torch.__version__

'1.5.0'

## Pytorch関係なく大前提として

- 最近のOSSはドキュメント、ソースコードともにある程度読める
- 何をしいていかわからない、うまく動かない場合は
  - `エラーメッセージ`を見つつ、ドキュメントで仕様を確認し、ソースコードを含めエラーがないか確認すれば解決します。
  - もちろん、聞いてくれるのは大歓迎なのですが、エンジニアリング力をあげるという意味では上のことができるようになった方がよいです。

- 見るべきもの
  - Pytorchのドキュメント: https://pytorch.org/docs/stable/index.html
  - Pythonのドキュメント: https://docs.python.org/ja/3/

## NNを楽に書くために必要なもの
- Pythonに対する理解
- Pytorchに対する理解
- 機械学習に対する理解

## NNライブラリを理解する上で難しいもの
- 機械学習側もある程度理解しないとイメージしづらい
  - 何をしているか
  - なぜそそうているか

## 今回の紹介内容
- Pytorchのnn等の中身について

__注意事項__

- どこが足りないか(Python?Pytoch?機械学習)を見つつ理解を楽しんでくれれば
- Pytorchの内部に近い挙動を見ていくつもりなので、かなり難しいです。
- 触りつつ、エラーメッセージみつつ、動作させてみて、理解を深めましょう

## Pytorchのモジュール構成(一部抜粋)
```
.
├── autograd #自動微分系のもの
├── backends #backend cuda/cudnn/mki etc
├── contrib # 現在はtensorboardしかなかった?
├── cuda # cudaはここも
├── distributed # 分散コンピューティング用
├── distributions # 確率分布
├── for_onnx #onnx用モジュール ライブラリに依存しないNNのformat こっちは現状ほぼ中身がない
├── jit # jit compile用
├── lib # C++用
├── multiprocessing # マルチプロセス用
├── nn # nnのモジュール
├── onnx # onnnx用モジュール
├── optim # 最適化のモジュール
├── quantization # 量子化
├── sparse # スパース行列用
├── testing # テスト用の準備
├── utils # 便利ツール data, tensorboard等
```

大きく分けて2種類
- 高速化・共通化・拡大化のためにあるもの(今回は触れない. CSの知識が必要)
- NNの計算にはあるとよいもの(今回話すもの)
  - Pythonレベルで知っておいてほしいもの

## Pytorchで知っておくとよいもの
- データの処理
  - `torch.utils.data`: 前回話した
- パラメータ更新の機能
  - `optim`: 次回話すかも
- ネットワークを保持するクラス(今日の話)
  - nn.Parameters
  - nn.Module
  - nn.Linear
  - nn.ModuleList
  - nn.Sequential

## 基本的なPytorchのネットワークの作り方
- 基本的な要素(行列の掛け算等)はPytorch側で用意
  - ネットワークの一層、活性化関数
  - 非常によく使われるNN(ResNet,LSTM)
- それを組み合わせて新しネットワークのクラスを作成
- ネットワワークのインスタンス `net`に対しては以下の操作ができる
  - forward演算: 行列、活性化関数などをかける。 
    - `net(x)`
  - backward演算: 微分の計算
    - `loss.backward()`
- プログラムを書く時は、以下さえできればよい
  - ネットワークの定義
  - forward演算の結果
  - 損失の計算

```python
class Net(nn.Module):
    def __init__(self, hoge, fuga, ....):
        super()__init__
        self.network1 = hoge
        self.network2 = fuga
        
    def forward(self, x):
        x = self.network1(x)
        x = self.network2(x)
        ...
        retrun self.network_last(x)
```

## ネットワークを実現するために
1. パラメータとそれ以外の区別
  - `nn.Parametes`
2. モジュールを再帰的な構造にしている
  - `nn.Module`
3. (裏で実際に計算で利用した計算グラフの取得)
  - 今回は紹介しない

## 楽な計算をするために
- 複数データ、1データでも同じ結果になるようにする

## 具体的なNNを見ていくためにPythonの基本的な道具について解説する

- Pythonのクラスの初期化
- OrderDict
- generator
- lambda 式


### Pythonのクラス定義における `__init__`
- クラスの初期化の時に実行されるもの
- 以下に例を記述



In [86]:
class InitSample():
    def __init__(self, hoge):
        print("class initialized")
        self.hoge = hoge

init_sample = InitSample("fuga")
init_sample.hoge

class initialized


'fuga'


## Pythonでクラスのインスタンスを作成する場合(参考)
1. `__new__` : クラス全体を見ながらインスタンス(`self`)作成
2. `__init__`: インスタンス(`self`)の情報を初期化する

あくまでそういう方針で作っているだけなので`__new__`で全部作れてしまう。
ただ、ベースがそういう考え方であることは理解しておいてください

## `__setattr__` (Pythonのメソッド)
  `object.__setattr__(self, name, value)`
  属性の代入が試みられた際に呼び出される
  通常の代入の過程 (すなわち、インスタンス辞書への値の代入) の代わりに呼び出される。name は属性名で、value はその属性に代入する値です。

In [182]:
# setattrのサンプル
class SetAttrSample():
    def __setattr__(self, name, value):
        print("name", name)
        print("value", value)
        
sample = SetAttrSample()
sample.x = "0"

name x
value 0


In [183]:
sample.x # 値が定義されてないのでエラーになる

AttributeError: 'SetAttrSample' object has no attribute 'x'

In [185]:
# setattrのサンプル
class SetAttrSample2():
    def __setattr__(self, name, value):
        print("name", name)
        print("value", value)
        object.__setattr__(self, name, value)
        # self.__setattr__(self, name, value)
        # self[name] = valeu だと無限ループになることに注意

sample2 = SetAttrSample2()
sample2.x = "0"
sample2.x

name x
value 0


'0'

### 上で出てきたobjectとは
object は全てのクラスの基底クラス
Pythonのクラスの全てのインスタンスに共通のメソッド群を持つ

- 今回の場合:
  - 親クラスの`__setattr__`メソッドを実行したことに相当

### クラスの実行`__call__`:
- `__call__`: が定義されていた場合
  - net(x)のようだ動作ができる
- 例

In [200]:
class A:
    def __call__(self, x):
        print(x)
a = A()
a(3)

3


### OrderDict(Pythonのモジュール)
- 順序を保つDict
- 順序を保つのでforループで同じ順番でkeyが得られる
- collectionsモジュールの一つ
- 最近のpythonではdictでも順序を保持している

In [4]:
# 例
from collections import OrderedDict
od = OrderedDict()

od['k1'] = 1
od['k2'] = 2
od['k3'] = 3

for key, val in od.items():
    print("key", key)
    print("val", val)

key k1
val 1
key k2
val 2
key k3
val 3


In [17]:
len([od])

1

## generator
- イテレータの一種で、要素を一つ取得する時に処理を加えて要素を渡すもの
- Pythonではyield文を使った実装が多い
- yieldはその値を取得して途中の結果が得られるもの

In [5]:
def my_generator(x):
    yield x+1
    yield x+2
    yield x+3

gen = my_generator(3)
print(type(gen))

for g in gen:
    print(g)

<class 'generator'>
4
5
6


In [13]:
def plus_one_generator(x):
    for _x in x:
        yield _x + 1

def plus_one_list(x):
    return [_x + 1 for _x in x]

In [14]:
%timeit plus_one_generator(list(range(100)))

1.18 µs ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
%timeit plus_one_list(list(range(100)))

4.95 µs ± 33.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## lambda式
- 無名関数ともいう
- 書き方: `lambda  引数 :演算`
- 演算結果が得られる

In [191]:
f = lambda x: x ** 2
f(3)

9

### nn.Parameters

In [179]:
class Parameter(torch.Tensor):
    r"""A kind of Tensor that is to be considered a module parameter.
    Parameters are :class:`~torch.Tensor` subclasses, that have a
    very special property when used with :class:`Module` s - when they're
    assigned as Module attributes they are automatically added to the list of
    its parameters, and will appear e.g. in :meth:`~Module.parameters` iterator.
    Assigning a Tensor doesn't have such effect. This is because one might
    want to cache some temporary state, like last hidden state of the RNN, in
    the model. If there was no such class as :class:`Parameter`, these
    temporaries would get registered too.
    Arguments:
        data (Tensor): parameter tensor.
        requires_grad (bool, optional): if the parameter requires gradient. See
            :ref:`excluding-subgraphs` for more details. Default: `True`
    """
    # インスタンスを作成するメソッド
    def __new__(cls, data=None, requires_grad=True):
        if data is None:
            data = torch.Tensor()
        return torch.Tensor._make_subclass(cls, data, requires_grad)

    # deepcopy実行時の挙動の指定
    def __deepcopy__(self, memo):
        if id(self) in memo:
            return memo[id(self)]
        else:
            result = type(self)(self.data.clone(memory_format=torch.preserve_format), self.requires_grad)
            memo[id(self)] = result
            return result
    
    # repr()実行時にパラメータであることを明記
    def __repr__(self):
        return 'Parameter containing:\n' + super(Parameter, self).__repr__()
    
    # pickleにする時に必要, hookを初期化している
    def __reduce_ex__(self, proto):
        # See Note [Don't serialize hooks]
        return (
            torch._utils._rebuild_parameter,
            (self.data, self.requires_grad, OrderedDict())
        )

### Parameterのまとめ
- Parameter固有のメソッド類は少ない
- ただし、Parameterとわかるようにrepr等が新しく設定されている
- 使われ方はNNのパラメータかどうか、パラメータとパラメータ以外のtensorを区別するために別クラスを定義した

## 例

In [141]:
import torch.nn.functional as F

In [142]:
class SimpleLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(SimpleLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
    
    def forward(self, x):
        return F.linear(x, self.weight)

In [145]:
l = SimpleLinear(3, 3)
l(torch.tensor([1. , 2., 3.]))

tensor([7.3787e+19, 1.5849e+29, 0.0000e+00], grad_fn=<SqueezeBackward3>)

## nn.Module
- pytorchのNNを実行(forward)/微分(backward)する親玉のクラス
- https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/module.py

コメント
```
Base class for all neural network modules.
Your models should also subclass this class.
```

## nn.Moduleのソースコード
- それなりに大きいので、一部基本的なものだけみる
- メソッド単位で分割する
- 実際に何が起きてるかは実験しつつ確認する

## `__init__` の処理

### nn.Moduleの`__init__`

In [83]:
from collections import OrderedDict

class Module:
    def __init__(self) -> None:
        """
        Initializes internal Module state, shared by both nn.Module and ScriptModule.
        """
        torch._C._log_api_usage_once("python.nn_module")

        self.training = True
        # self._non_persistent_buffers_set = set() 最新のgithub上には存在()
        self._parameters = OrderedDict()
        self._buffers = OrderedDict()
        self._backward_hooks = OrderedDict()
        self._forward_hooks = OrderedDict()
        self._forward_pre_hooks = OrderedDict()
        self._state_dict_hooks = OrderedDict()
        self._load_state_dict_pre_hooks = OrderedDict()
        self._modules = OrderedDict()

## 無視するもの
- hook系のもの
- hookはある関数が実行されたときに補正処理をおこなもののこと
  - forward_hooksの場合forward演算で補正がしたい場合の処理一覧が登録
  - backward_hooksの場合…

## Pytorchの単純なクラスで__init__で定義された値がどうなったかを確認

In [150]:
n = nn.Linear(3, 4)
n._parameters
# Parameter containing:はParameterクラスの __repr__で加えられたもの

OrderedDict([('weight',
              Parameter containing:
              tensor([[ 0.3474,  0.4222, -0.0881],
                      [-0.3853, -0.4168, -0.0868],
                      [-0.3814, -0.4686,  0.1939],
                      [ 0.0700,  0.3962, -0.5166]], requires_grad=True)),
             ('bias',
              Parameter containing:
              tensor([ 0.0721,  0.4652, -0.4289, -0.3902], requires_grad=True))])

## 後は定義されておらず

In [149]:
print(n._buffers)
print(n._backward_hooks)
print(n._forward_hooks)
print(n._forward_pre_hooks)
print(n._state_dict_hooks)
print(n._load_state_dict_pre_hooks)
print(n._modules)

OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()


### よくあるネットワーク定義の場合

In [68]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        print("after super modules", self._modules)
        self.l1 = nn.Linear(3, 4)
        print("after l1 modules", self._modules)
        self.l2 = nn.Linear(4,3)
        print("after l2 modules", self._modules)
        self.l3 = "a"
        print("after l3 modules", self._modules)
    
    def forward(self, x):
        y = self.l1(x)
        return self.l2(y)

In [69]:
net = Net()

In [90]:
print(net._parameters)
print(net._buffers)
print(net._backward_hooks)
print(net._forward_hooks)
print(net._forward_pre_hooks)
print(net._state_dict_hooks)
print(net._load_state_dict_pre_hooks)

OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()
OrderedDict()


In [91]:
print(net._modules)

OrderedDict([('l1', Linear(in_features=3, out_features=4, bias=True)), ('l2', Linear(in_features=4, out_features=3, bias=True))])


### 注意
- Netにパラメータがないという意味ではない。
  - _parameterはどのクラスで定義されたかを管理
  - nn.Modulerのパラメータは parameters()で把握できる

In [153]:
for p in net.parameters():
    print(p)

Parameter containing:
tensor([[ 0.1978,  0.4489,  0.2995],
        [-0.5384, -0.1922, -0.2038],
        [ 0.5301,  0.2694, -0.4621],
        [ 0.3995, -0.4096,  0.3189]], requires_grad=True)
Parameter containing:
tensor([ 0.0071, -0.0110, -0.3840,  0.0931], requires_grad=True)
Parameter containing:
tensor([[ 0.4092,  0.2749, -0.2811, -0.1578],
        [ 0.2796,  0.1409, -0.4183, -0.4621],
        [-0.4077,  0.3120, -0.0261, -0.3705]], requires_grad=True)
Parameter containing:
tensor([-0.4393, -0.1938,  0.1679], requires_grad=True)


### netの作成時で気になるところ
- `__init__` しか実行していいはずだが、いつの間にか `._modules`が更新されている

## nn.Moduleの`__setattr__`

In [158]:
    def __setattr__(self, name, value):
        """
           1. valueがParatemerクラスの場合:
              register_parameterを実行
           2. valueがModuleクラスの場合:
              modulesに値を追加
           3. それ以外はbufferとして処理
        """
        # 内部用のメソッド定義
        def remove_from(*dicts):
            # 複数の引数をまとめてlistのように扱う
            for d in dicts:
                if name in d:
                    del d[name]

        params = self.__dict__.get('_parameters')
        
        # valueがParameterの場合
        if isinstance(value, Parameter):
            if params is None:
                raise AttributeError(
                    "cannot assign parameters before Module.__init__() call")
            remove_from(self.__dict__, self._buffers, self._modules)
            self.register_parameter(name, value)
            
        # Parameter名とnameがかぶった場合
        # valueがNoneの場合はパラメータを削除する
        elif params is not None and name in params:
            if value is not None:
                raise TypeError("cannot assign '{}' as parameter '{}' "
                                "(torch.nn.Parameter or None expected)"
                                .format(torch.typename(value), name))
            self.register_parameter(name, value)
        else:
            # valueがModuleの場合
            modules = self.__dict__.get('_modules')
            if isinstance(value, Module):
                if modules is None:
                    raise AttributeError(
                        "cannot assign module before Module.__init__() call")
                remove_from(self.__dict__, self._parameters, self._buffers)
                modules[name] = value
            elif modules is not None and name in modules:
                if value is not None:
                    raise TypeError("cannot assign '{}' as child module '{}' "
                                    "(torch.nn.Module or None expected)"
                                    .format(torch.typename(value), name))
                modules[name] = value
            else:  
                # valueがそれ以外の場合
                buffers = self.__dict__.get('_buffers')
                if buffers is not None and name in buffers:
                    if value is not None and not isinstance(value, torch.Tensor):
                        raise TypeError("cannot assign '{}' as buffer '{}' "
                                        "(torch.Tensor or None expected)"
                                        .format(torch.typename(value), name))
                    buffers[name] = value
                else:
                    # bufferでもない場合
                    # buffersがNoneでない場合必ずここにこないのは注意
                    # bufferでも,moduleでも,parameterでもないクラスは使えない
                    object.__setattr__(self, name, value)

## `__setattr__`の書き換え
- 長過ぎるので短くする
- 3要素に応じて処理を書き換える

In [126]:
def __setattr__(self, name, value):
    """
       1. valueがParatemerクラスの場合:
          register_parameterを実行
       2. valueがModuleクラスの場合:
          modulesに値を追加
       3. それ以外はbufferとして処理
    """
    # 内部用のメソッド定義
    def remove_from(*dicts):
        # 複数の引数をまとめてlistのように扱う
        for d in dicts:
            if name in d:
                del d[name]
    # 1. 
    parameter_setted = __parameter__setattr(self, name, value, remove_from)
    if not parameter_setted:
        # 2. 
        module_setted = __module_settattr(self, name, value, remove_from)
        if not module_setted:
            # 3.
            __buffer_setattr(self, name, value, remove_from)

### 1. `__parameter__setattr`
- valueがパラメータクラスか確認
  - パラメータクラスの場合,パラメータを保存する変数が値が追加できる状態か確認し,
    parameterに値を追加する
  - パラメータクラスでない場合,
    parameterに登録された名前と同じnameかつvalueがNoneの時はparmeter削除と判定し、削除処理を実行
    それ以外の変な状況はエラーにする

In [119]:
def __parameter__setattr(self, name, value, remove_from):
    params = self.__dict__.get('_parameters')
    if isinstance(value, Parameter):
            if params is None:
                raise AttributeError(
                    "cannot assign parameters before Module.__init__() call")
            remove_from(self.__dict__, self._buffers, self._modules)
            self.register_parameter(name, value)
            return True
            
        # Parameter名とnameがかぶった場合
        # valueがNoneの場合はパラメータを削除する
    elif params is not None and name in params:
        if value is not None:
            raise TypeError("cannot assign '{}' as parameter '{}' "
                                "(torch.nn.Parameter or None expected)"
                                .format(torch.typename(value), name))
        self.register_parameter(name, value)
        return True
    else:
        return False

### 2. `__module__setattr`

In [123]:
def __module__setattr(self, name, value, remove_from):
    # valueがModuleの場合
    modules = self.__dict__.get('_modules')
    if isinstance(value, Module):
        if modules is None:
            raise AttributeError(
                "cannot assign module before Module.__init__() call")
        remove_from(self.__dict__, self._parameters, self._buffers)
        modules[name] = value
        return True
    elif modules is not None and name in modules:
        if value is not None:
            raise TypeError("cannot assign '{}' as child module '{}' "
                            "(torch.nn.Module or None expected)"
                            .format(torch.typename(value), name))
        modules[name] = value
        return True
    else:
        return False

### 3. `__buffer__setattr`

In [192]:
def __buffer__setattr(self, name, value, remove_from):
    # valueがそれ以外の場合
    buffers = self.__dict__.get('_buffers')
    if buffers is not None and name in buffers:
        if value is not None and not isinstance(value, torch.Tensor):
            raise TypeError("cannot assign '{}' as buffer '{}' "
                            "(torch.Tensor or None expected)"
                            .format(torch.typename(value), name))
        buffers[name] = value
    else:
        # bufferでもない場合
        # buffersがNoneでない場合必ずここにこないのは注意
        # bufferでも,moduleでも,parameterでもないクラスは使えない
        object.__setattr__(self, name, value)

### `__setattr__`に存在する三要素
- parameter: NNのパラメータ(実際はnn.Parameterクラスのオブジェクト)
- module: nn.Moduleのオブジェクト(Parameterも持つ)
- buffer: パラメータにしないTensor
- moduleと判定されるかParameterと判定されるかの違い
  - 自分のクラスがParameterを持つか
  - 自分のattributeがparameterを持つか

### パラメータの設定詳細
- やっている操作
  - パラメータクラスかの確認
    - `isinstance(object, class)` は、第一引数のオブジェクトが、第二引数の型のインスタンス、またはサブクラスのインスタンスであればTrueを返す関数
  - 条件に応じて`register_parameter`を実行しパラメータの追加

In [156]:
# 普通のモジュールがパラメータかどうかの確認
l = nn.Linear(3, 4)
isinstance(l, nn.Parameter)

False

### パラメータの追加

```python
def register_parameter(self, name, param):
    r"""Adds a parameter to the module.

    The parameter can be accessed as an attribute using given name.

    Args:
        name (string): name of the parameter. The parameter can be accessed
            from this module using the given name
        param (Parameter): parameter to be added to the module.
    """
    
    # パラメーターが登録削除できるかの確認(ここから)
    if '_parameters' not in self.__dict__:
        raise AttributeError(
            "cannot assign parameter before Module.__init__() call")

    elif not isinstance(name, torch._six.string_classes):
        raise TypeError("parameter name should be a string. "
                        "Got {}".format(torch.typename(name)))
    elif '.' in name:
        raise KeyError("parameter name can't contain \".\"")
    elif name == '':
        raise KeyError("parameter name can't be empty string \"\"")
    elif hasattr(self, name) and name not in self._parameters:
        raise KeyError("attribute '{}' already exists".format(name))
    # 異常状態の確認(ここから)
    
    # パラメータの削除
    if param is None:
        self._parameters[name] = None
    
    # パラメータが登録できるかどうかの確認
    elif not isinstance(param, Parameter):
        raise TypeError("cannot assign '{}' object to parameter '{}' "
                        "(torch.nn.Parameter or None required)"
                        .format(torch.typename(param), name))
    elif param.grad_fn:
        raise ValueError(
            "Cannot assign non-leaf Tensor to parameter '{0}'. Model "
            "parameters must be created explicitly. To express '{0}' "
            "as a function of another Tensor, compute the value in "
            "the forward() method.".format(name))
    else:
        # パラメータの登録
        self._parameters[name] = param
```

### `register_parameter`でやっていること
- 例外キャッチ多数
- 値代入、削除
- 実質は以下
```python
    if param is None:
        self._parameters[name] = None
    else:
        # パラメータの登録
        self._parameters[name] = param
```

### モジュールの設定詳細
- Moduleかどうかの判定
- Moduleへの代入

## 普段扱っているモジュールはnn.Moduleのインスタンス

In [160]:
isinstance(n, nn.Module) ## nn.Moduleを継承しているか

True

In [161]:
isinstance(net, nn.Module)

True

Moduleの場合は`_modules`に辞書的に追加
```python
modules[name] = value
```

## Bufferの代入削除
- Moduleと同様

## `nn.Module`のパラメータ確認方法
- _parametesは下のクラスで定義されたものは入っていない
- ただし、以下等で確認できる

In [165]:
for name, parame in net.named_parameters():
    print(name, parame)

l1.weight Parameter containing:
tensor([[ 0.1978,  0.4489,  0.2995],
        [-0.5384, -0.1922, -0.2038],
        [ 0.5301,  0.2694, -0.4621],
        [ 0.3995, -0.4096,  0.3189]], requires_grad=True)
l1.bias Parameter containing:
tensor([ 0.0071, -0.0110, -0.3840,  0.0931], requires_grad=True)
l2.weight Parameter containing:
tensor([[ 0.4092,  0.2749, -0.2811, -0.1578],
        [ 0.2796,  0.1409, -0.4183, -0.4621],
        [-0.4077,  0.3120, -0.0261, -0.3705]], requires_grad=True)
l2.bias Parameter containing:
tensor([-0.4393, -0.1938,  0.1679], requires_grad=True)


In [166]:
for p in net.parameters():
    print(p)

Parameter containing:
tensor([[ 0.1978,  0.4489,  0.2995],
        [-0.5384, -0.1922, -0.2038],
        [ 0.5301,  0.2694, -0.4621],
        [ 0.3995, -0.4096,  0.3189]], requires_grad=True)
Parameter containing:
tensor([ 0.0071, -0.0110, -0.3840,  0.0931], requires_grad=True)
Parameter containing:
tensor([[ 0.4092,  0.2749, -0.2811, -0.1578],
        [ 0.2796,  0.1409, -0.4183, -0.4621],
        [-0.4077,  0.3120, -0.0261, -0.3705]], requires_grad=True)
Parameter containing:
tensor([-0.4393, -0.1938,  0.1679], requires_grad=True)


In [197]:
net.state_dict()
# state_dictについては今回はこれ以上紹介しない

OrderedDict([('l1.weight',
              tensor([[ 0.1978,  0.4489,  0.2995],
                      [-0.5384, -0.1922, -0.2038],
                      [ 0.5301,  0.2694, -0.4621],
                      [ 0.3995, -0.4096,  0.3189]])),
             ('l1.bias', tensor([ 0.0071, -0.0110, -0.3840,  0.0931])),
             ('l2.weight',
              tensor([[ 0.4092,  0.2749, -0.2811, -0.1578],
                      [ 0.2796,  0.1409, -0.4183, -0.4621],
                      [-0.4077,  0.3120, -0.0261, -0.3705]])),
             ('l2.bias', tensor([-0.4393, -0.1938,  0.1679]))])

## parametersの実装
- yieldで必要な情報を加工したgeneratorを作成
- 実質必要な情報は_named_membersから取得し、結果を返却

In [188]:
    def parameters(self, recurse=True):
        for name, param in self.named_parameters(recurse=recurse):
            yield param

    def named_parameters(self, prefix='', recurse=True):
        gen = self._named_members(
            # モジュールに対し、そのモジュールの持つパラメータ一覧を取得
            lambda module: module._parameters.items(),
            prefix=prefix, recurse=recurse)
        # モジュールごとにモジュールの持つパラメータ一覧を返却する
        for elem in gen:
            yield elem

In [193]:
# モジュールの一覧からget_members_fnで必要な情報一覧を取得する
# どこから取得したかわかるようにmodule_prefix等で補正する
def _named_members(self, get_members_fn, prefix='', recurse=True):
    r"""Helper method for yielding various names + members of modules."""
    memo = set()
    modules = self.named_modules(prefix=prefix) if recurse else [(prefix, self)]
    for module_prefix, module in modules:
        members = get_members_fn(module)
        for k, v in members:
            if v is None or v in memo:
                continue
            memo.add(v)
            name = module_prefix + ('.' if module_prefix else '') + k
            yield name, v

# モジュール一覧の取得
def named_modules(self, memo=None, prefix=''):
    if memo is None:
        memo = set()
    if self not in memo:
        # 自分自信を追加
        memo.add(self)
        yield prefix, self
        # 自分のモジュール一覧を取得
        for name, module in self._modules.items():
            if module is None:
                continue
            # モジュールのprefixを追加
            submodule_prefix = prefix + ('.' if prefix else '') + name
            for m in module.named_modules(memo, submodule_prefix):
                yield m


## 関係を改めてまとめる
- parameters():
  - パラメータを返すジェネレーター
  - 実装上はnamed_parameters（）から名前を捨てただけ
- named_parametes():
  - パラメータの名前と値を返すジェネレータ
  - 実際はモジュール一覧から` lambda module: module._parameters.items(),`で処理をした情報を受けとっただけ
- _named_members():
  - モジュールの名前と`get_members_fn`で選択した情報を返すジェネレータ
  - モジュール一覧は`named_modules`から取得
- named_modules():
  - モジュール一覧の取得
  - モジュールの名前でネストした名前をとモジュールの組を返す

### クラスの実行
- 処理は
  - hookに基づき入力を変更
  - forwardの実行
  - forward結果に対するhookの実行
  - grad_fnの登録(計算グラフのデバッグ用)

In [55]:
    def __call__(self, *input, **kwargs):
        for hook in self._forward_pre_hooks.values():
            result = hook(self, input)
            if result is not None:
                if not isinstance(result, tuple):
                    result = (result,)
                input = result
        if torch._C._get_tracing_state():
            result = self._slow_forward(*input, **kwargs)
        else:
            result = self.forward(*input, **kwargs)
        for hook in self._forward_hooks.values():
            hook_result = hook(self, input, result)
            if hook_result is not None:
                result = hook_result
        if len(self._backward_hooks) > 0:
            var = result
            while not isinstance(var, torch.Tensor):
                if isinstance(var, dict):
                    var = next((v for v in var.values() if isinstance(v, torch.Tensor)))
                else:
                    var = var[0]
            grad_fn = var.grad_fn
            if grad_fn is not None:
                for hook in self._backward_hooks.values():
                    wrapper = functools.partial(hook, self)
                    functools.update_wrapper(wrapper, hook)
                    grad_fn.register_hook(wrapper)
        return result

## backwardの計算
- torch.autograd.backwardで行う
- 詳細はCレイヤなので省略
- 内部的にはfunctionが前のfunctionを覚えていて、それでどうにかする形
- `loss.grad_fn.next_functions`をすると一つ前の関数がわかる

### 重要なポイント
- 計算グラフ情報は`nn.Module`が覚えているわけではない
- これによって`nn.Module`を複数回使うことも容易になる(RNN等)

## `nn.Module`まとめ
- parameterやmoduleをbufferを設定できる
- moduleによる再帰構造が可能
- forwardを定義すればそれに基づき計算する
- backwardはnn.Module内には存在しない

## nn.Moduleの典型例
- nn.Sequnetial
- nn.ModuleList
- nn.Liner

## nn.Sequential
- Sequentialクラスにmoduleを加える
- forwardでそれを実行する

In [20]:
from torch._jit_internal import _copy_to_script_wrapper

class Sequential(nn.Module):
    r"""A sequential container.
    Modules will be added to it in the order they are passed in the constructor.
    Alternatively, an ordered dict of modules can also be passed in.

    To make it easier to understand, here is a small example::

        # Example of using Sequential
        model = nn.Sequential(
                  nn.Conv2d(1,20,5),
                  nn.ReLU(),
                  nn.Conv2d(20,64,5),
                  nn.ReLU()
                )

        # Example of using Sequential with OrderedDict
        model = nn.Sequential(OrderedDict([
                  ('conv1', nn.Conv2d(1,20,5)),
                  ('relu1', nn.ReLU()),
                  ('conv2', nn.Conv2d(20,64,5)),
                  ('relu2', nn.ReLU())
                ]))
    """

    def __init__(self, *args):
        super(Sequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict):
            for key, module in args[0].items():
                self.add_module(key, module)
        else:
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)

    def _get_item_by_idx(self, iterator, idx):
        """Get the idx-th item of the iterator"""
        size = len(self)
        idx = operator.index(idx)
        if not -size <= idx < size:
            raise IndexError('index {} is out of range'.format(idx))
        idx %= size
        return next(islice(iterator, idx, None))

    @_copy_to_script_wrapper
    # itemを取得
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            return self.__class__(OrderedDict(list(self._modules.items())[idx]))
        else:
            return self._get_item_by_idx(self._modules.values(), idx)

    # itemを変更
    def __setitem__(self, idx, module):
        key = self._get_item_by_idx(self._modules.keys(), idx)
        return setattr(self, key, module)
    
    # itemを削除
    def __delitem__(self, idx):
        if isinstance(idx, slice):
            for key in list(self._modules.keys())[idx]:
                delattr(self, key)
        else:
            key = self._get_item_by_idx(self._modules.keys(), idx)
            delattr(self, key)

    @_copy_to_script_wrapper
    # 長さを取得
    def __len__(self):
        return len(self._modules)

    @_copy_to_script_wrapper
    def __dir__(self):
        keys = super(Sequential, self).__dir__()
        keys = [key for key in keys if not key.isdigit()]
        return keys

    @_copy_to_script_wrapper
    def __iter__(self):
        return iter(self._modules.values())

    def forward(self, input):
        for module in self:
            input = module(input)
        return input

In [21]:
seq = Sequential(nn.Linear(2,3))

In [22]:
for mod in seq:
    print(mod)

Linear(in_features=2, out_features=3, bias=True)


In [213]:
seq.add_module("l2", nn.Linear(3,5))
# operator等が定義されていないので動作しないものもある

In [212]:
seq

Sequential(
  (0): Linear(in_features=2, out_features=3, bias=True)
  (l2): Linear(in_features=3, out_features=5, bias=True)
)

## Seqeuntialのポイント
- forwardが登録した順になる
- ネットワークが分岐するような特殊なものは作れない

## nn.ModuleList
- moduleのListを管理
- forwardはサポートせず、自分で各moduleごとに実行する

### 特徴
- iadd (+=)が実装
- appendが実装
- extendが実装
- forwardが実装されず

In [217]:
class ModuleList(nn.Module):
    r"""Holds submodules in a list.

    :class:`~torch.nn.ModuleList` can be indexed like a regular Python list, but
    modules it contains are properly registered, and will be visible by all
    :class:`~torch.nn.Module` methods.

    Arguments:
        modules (iterable, optional): an iterable of modules to add

    Example::

        class MyModule(nn.Module):
            def __init__(self):
                super(MyModule, self).__init__()
                self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

            def forward(self, x):
                # ModuleList can act as an iterable, or be indexed using ints
                for i, l in enumerate(self.linears):
                    x = self.linears[i // 2](x) + l(x)
                return x
    """

    def __init__(self, modules=None):
        super(ModuleList, self).__init__()
        if modules is not None:
            self += modules

    def _get_abs_string_index(self, idx):
        """Get the absolute index for the list of modules"""
        idx = operator.index(idx)
        if not (-len(self) <= idx < len(self)):
            raise IndexError('index {} is out of range'.format(idx))
        if idx < 0:
            idx += len(self)
        return str(idx)

    @_copy_to_script_wrapper
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            return self.__class__(list(self._modules.values())[idx])
        else:
            return self._modules[self._get_abs_string_index(idx)]

    def __setitem__(self, idx, module):
        idx = self._get_abs_string_index(idx)
        return setattr(self, str(idx), module)

    def __delitem__(self, idx):
        if isinstance(idx, slice):
            for k in range(len(self._modules))[idx]:
                delattr(self, str(k))
        else:
            delattr(self, self._get_abs_string_index(idx))
        # To preserve numbering, self._modules is being reconstructed with modules after deletion
        str_indices = [str(i) for i in range(len(self._modules))]
        self._modules = OrderedDict(list(zip(str_indices, self._modules.values())))

    @_copy_to_script_wrapper
    def __len__(self):
        return len(self._modules)

    @_copy_to_script_wrapper
    def __iter__(self):
        return iter(self._modules.values())

    def __iadd__(self, modules):
        return self.extend(modules)

    @_copy_to_script_wrapper
    def __dir__(self):
        keys = super(ModuleList, self).__dir__()
        keys = [key for key in keys if not key.isdigit()]
        return keys

    def insert(self, index, module):
        r"""Insert a given module before a given index in the list.

        Arguments:
            index (int): index to insert.
            module (nn.Module): module to insert
        """
        for i in range(len(self._modules), index, -1):
            self._modules[str(i)] = self._modules[str(i - 1)]
        self._modules[str(index)] = module

    def append(self, module):
        r"""Appends a given module to the end of the list.

        Arguments:
            module (nn.Module): module to append
        """
        self.add_module(str(len(self)), module)
        return self

    def extend(self, modules):
        r"""Appends modules from a Python iterable to the end of the list.

        Arguments:
            modules (iterable): iterable of modules to append
        """
        # container_abcs.Iterableがここでは定義されてないためエラーになる
        if not isinstance(modules, container_abcs.Iterable):
            raise TypeError("ModuleList.extend should be called with an "
                            "iterable, but got " + type(modules).__name__)
        offset = len(self)
        for i, module in enumerate(modules):
            self.add_module(str(offset + i), module)
        return self

    def forward(self):
        raise NotImplementedError()


In [218]:
nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

ModuleList(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Linear(in_features=10, out_features=10, bias=True)
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Linear(in_features=10, out_features=10, bias=True)
  (4): Linear(in_features=10, out_features=10, bias=True)
  (5): Linear(in_features=10, out_features=10, bias=True)
  (6): Linear(in_features=10, out_features=10, bias=True)
  (7): Linear(in_features=10, out_features=10, bias=True)
  (8): Linear(in_features=10, out_features=10, bias=True)
  (9): Linear(in_features=10, out_features=10, bias=True)
)

## nn.Linear
- weightをパラメータに
- biasをパラメータに
- weightとbiasの初期値を設定

In [223]:
class Linear(nn.Module):
    __constants__ = ['in_features', 'out_features']

    def __init__(self, in_features, out_features, bias=True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    # パラメータの初期値を設定
    def reset_parameters(self):
        # kaiming_uniformと呼ばれる一様分布でパラメータを初期化
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        # nn.functionalと呼ばれる演算の定義されたクラス
        # 内部は実際はCを呼ぶ
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )

## Loss関数を定義する
- 損失はbufferに登録
- 実際の計算は`F.cross_entropy`:
  - これもC側の実装
  - weightは特定の項目の重み付けを変更するもの

In [226]:
class _Loss(Module):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction

class _WeightedLoss(_Loss):
    def __init__(self, weight=None, size_average=None, reduce=None, reduction='mean'):
        super(_WeightedLoss, self).__init__(size_average, reduce, reduction)
        # 損失はparameterではないのでbufferに登録
        self.register_buffer('weight', weight)


class CrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, size_average=None, ignore_index=-100,
                 reduce=None, reduction='mean'):
        super(CrossEntropyLoss, self).__init__(weight, size_average, reduce, reduction)
        self.ignore_index = ignore_index

    def forward(self, input, target):
        return F.cross_entropy(input, target, weight=self.weight,
                               ignore_index=self.ignore_index, reduction=self.reduction)


## Optimizer

In [229]:
import torch
from torch.optim.optimizer import Optimizer, required

class SGD(torch.optim.Optimizer):
    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(SGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad
                if weight_decay != 0:
                    d_p = d_p.add(p, alpha=weight_decay)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(d_p, alpha=1 - dampening)
                    if nesterov:
                        d_p = d_p.add(buf, alpha=momentum)
                    else:
                        d_p = buf

                p.add_(d_p, alpha=-group['lr'])

        return loss

## `__init__`
- パラメータの妥当性チェック
- 正常な時のパラメータ設定
```python
        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        super(SGD, self).__init__(params, defaults)
```

- self.param_groups:
  - groupの配列
  - groupはParameterクラスと`defaults`を加えた`dict`

## `step`
- 実質以下
  - 実際はmomentum,dampening,nesterov等SGDの亜種にするための設定を加える
  - weight_decay等のパラメータの設定も加える
```python
for p in group['params']:
    d_p = p.grad
    p.add_(d_p, alpha=-group['lr'])
```

## 改めて

- `net = nn.Module()` ニューラルネットワーク自体の定義
- `loss = criterion(net(x), y)` 損失の定義
- `loss.backward()`: 損失によるパラメータの微分
- `optimzer.step()`: optimizerに設定したパラメータの更新

後はこれをデータとEpoch数だけ繰り返せば良い

In [238]:
class Net(nn.Module):
    def __init__(self, networks):
        super(Net, self).__init__()
        self.networks = networks

    def forward(self, x):
        for net in self.networks:
            x = net(x)
            x = nn.ReLU()(x)
        return x
net = Net([nn.Linear(3,5), nn.Linear(5, 4)])
net(torch.tensor([1., 2., 3.]))

tensor([0.6504, 0.0000, 0.0000, 0.2218], grad_fn=<ReluBackward0>)

## 演習
以下のソースコードはそのままでは動かない.何箇所か修正し,活性化関数ReLUの3層のnnの計算を実行せよ.
```python
class Net(Module):
    def __init__(self, netwoerks):
        self.networks = networks

    def forward(self, x):
        for net in self.networks:
            net(x)
net = Net([nn.Linear(3,5), nn.Linear(5, 4)])
net(torch.tensor([1, 2, 3]))
```

## 誤り
- Moduleクラスが定義されていない(かもしれない)
- `super`が実行されていない
- 引数がnetwoerksになっている
- forwardでnetの結果が反映されていない
- 活性化関数が定義されていない
- forwardの結果がReturnされていない

In [239]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, networks):
        super(Net, self).__init__()
        self.networks = networks

    def forward(self, x):
        for net in self.networks:
            x = net(x)
            x = nn.ReLU()(x)
        return x
net = Net([nn.Linear(3,5), nn.Linear(5, 4)])
net(torch.tensor([1., 2., 3.]))

tensor([0.1999, 0.0000, 0.3270, 0.0000], grad_fn=<ReluBackward0>)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
    
    def forward(self, x):
        l1 = nn.Linear(4, 3)
        return l1(x)
    
net, x, y = Net(), torch.tensor([[1,2,3,4]]), torch.tensor([1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.01)
loss = net(x)
[loss.backward() for _ in range(100)]    

ValueError: optimizer got an empty parameter list

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(4, 3)
    
    def forward(self, x):
        return self.l1(x)
    
net, x, y = Net(), torch.tensor([[1,2,3,4]]).float(), torch.tensor([1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
for _ in range(100):
    optimizer.zero_grad()
    loss = criterion(net(x), y)
    loss.backward()
    optimizer.step()

In [30]:
net(x)

tensor([[-3.1160,  2.6083, -1.6311]], grad_fn=<AddmmBackward>)

## 誤り

In [31]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.nn_list = [nn.Linear(4,4), nn.Linear(4, 3)]
    
    def forward(self, x):
        for l in nn_list:
            x = nn.ReLU()(l(x))
        return x,
net, x, y = Net(), torch.tensor([1,2,3,4], [2,3,10, 1]), torch.tensor([1, 2])
criterion = nn.MSe()
optimizer = torch.optim.SGD(net.parameters())
for i in range(100):
    criterion(net(x), y).backward(); optimizer.step()

TypeError: tensor() takes 1 positional argument but 2 were given

## 訂正

In [60]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.nn_list = nn.ModuleList([nn.Linear(4,4), nn.Linear(4, 1)])
    
    def forward(self, x):
        for l in self.nn_list:
            x = nn.ReLU()(l(x))
        return x
net, x, y = Net(), torch.tensor([[1,2,3,4], [2,3,10, 1]]).float(), torch.tensor([1, 2]).float()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
for i in range(1000):
    optimizer.zero_grad()
    criterion(net(x)[0], y.view(-1, 1)[0]).backward()
    if i % 100 == 0:
        print(net(x))
    optimizer.step()

tensor([[0.5021],
        [1.2318]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)
tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)


In [61]:
net(x)

tensor([[1.0000],
        [1.7440]], grad_fn=<ReluBackward0>)

In [35]:
torch.tensor([[1,2,3,4], [2,3,10, 1]]).float()

tensor([[ 1.,  2.,  3.,  4.],
        [ 2.,  3., 10.,  1.]])

In [38]:
net(x)

tensor([[0.],
        [0.]], grad_fn=<ReluBackward0>)

In [39]:
y

tensor([1., 2.])

In [42]:
y.view(-1, 1)

tensor([[1.],
        [2.]])

In [43]:
net(x)

tensor([[1.7521],
        [1.7521]], grad_fn=<ReluBackward0>)

In [62]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3,5)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(5,4)

    def forward(self,x):
        return self.l2(self.relu(self.l1(torch.tensor[x])))

net = Net()
net.forward(torch.tensor[1,2,3])

TypeError: 'builtin_function_or_method' object is not subscriptable

## デバッグ

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.nn_list = [nn.Linear(4,4), nn.Linear(4, 3)]
        self.layer = nn.ModuleList(self.nn_list)
    def forward(self, x):
        for l in self.layer:
            x = F.relu(l(x))
        return x

net, x, y = Net(), torch.tensor([1.0,2.0,3.0,4.0]), torch.tensor([1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),0.01)


In [35]:
net(x)

tensor([0.0000, 2.5897, 0.0000], grad_fn=<ReluBackward0>)

In [34]:

for i in range(100):
    optimizer.zero_grad()
    outputs=net(x)
    criterion(outputs.view(1,-1), y).backward(); 
    optimizer.step()
print(torch.argmax(net(x)))

tensor(1, grad_fn=<NotImplemented>)


In [36]:
nn.ModuleList([nn.Linear(4,4), nn.Linear(4, 3)])[-1]

Linear(in_features=4, out_features=3, bias=True)

In [38]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    
  def forward(self, x):
    for net in self.networks:
      x = net(x)
    return x


net = Net()
net.networks = ([nn.Linear(3, 5), nn.ReLU(), nn.Linear(5, 4), nn.ReLU()])
net(torch.tensor([1., 2., 3.]))

tensor([0.2091, 0.0622, 0.0218, 0.4440], grad_fn=<ReluBackward0>)